In [22]:
import pandas as pd
from pycaret.regression import *
from tqdm import tqdm

In [41]:
train_df = pd.read_csv('checkpoint_train_numerical.csv', encoding = 'utf-8')
train_df.drop(columns = 'Unnamed: 0', inplace = True)

test_df = pd.read_csv('checkpoint_test_numerical.csv', encoding = 'utf-8')
test_df.drop(columns = 'Unnamed: 0', inplace = True)

In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [29]:
train_df.head()

,rating,userId,movieId,budget,overview,popularity,release_date,revenue,runtime,tagline,...,History,Music,Crime,Documentary,Mystery,Drama,Horror,Science Fiction,Comedy,TV Movie
0,0.4,10.0,1358.0,1.000000e+06,Karl Childers is a mentally disabled man who h...,8.457968,1996.0,2.444412e+07,135.0,A simple man A difficult choice,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.7,237.0,1544.0,1.565299e+07,Four years after Jurassic Parks genetically br...,0.788123,1997.0,3.195007e+07,129.0,Something has survived,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,54.0,373.0,8.000000e+06,When a promised job for Texan Michael fails to...,5.137903,1993.0,2.000000e+06,98.0,Where nothing is as it seems,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.8,11.0,2053.0,1.565299e+07,Wayne Szalinski is at it again But instead of ...,8.457968,1992.0,3.195007e+07,89.0,The BIG Laughs Start January 6th,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.6,183.0,2524.0,1.400000e+07,At the opening party of a colossal - but poorl...,7.469056,1974.0,3.195007e+07,137.0,The worlds tallest building is on fire You are...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# text_columns = ['overview', 'tagline', 'title', 'all_keywords', 'part_of_collection']
# tokenizer = Tokenizer()
# max_len = 100

# train_df[text_columns] = train_df[text_columns].astype(str)
# test_df[text_columns] = test_df[text_columns].astype(str)

# for col in tqdm(text_columns):
#     tokenizer.fit_on_texts(train_df[col])
#     train_df[col] = [pad_sequences(tokenizer.texts_to_sequences(x), maxlen = max_len, padding='post') for x in train_df[col]]
#     test_df[col] = [pad_sequences(tokenizer.texts_to_sequences(x), maxlen = max_len, padding='post') for x in test_df[col]]


100%|██████████| 5/5 [04:02<00:00, 48.42s/it] 


In [42]:
genre_columns =     ['Science Fiction', 'Crime', 'Thriller', 'Drama', 
                     'Horror', 'Romance', 'TV Movie', 'Action', 
                     'Adventure', 'War', 'Documentary', 'Family', 
                     'Fantasy', 'Music', 'Animation', 'Mystery', 
                     'Comedy', 'Western', 'Foreign', 'History']
for col in tqdm(genre_columns):
    try:
        train_df[genre_columns] = train_df[genre_columns].astype(int)
        test_df[genre_columns] = test_df[genre_columns].astype(int)
    except ValueError:
        pass

100%|██████████| 20/20 [00:00<00:00, 154.03it/s]


In [40]:
train_df[text_columns].head()

,overview,tagline,title,all_keywords,part_of_collection
0,"[[6911, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2030, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[306, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[2616, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,"[[9348, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2787, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[1783, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2047, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[6842, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,"[[12454, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[13019, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[4707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[519, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[2616, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,"[[12454, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[2635, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[4697, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[3124, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[4576, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,"[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2635, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[1783, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[519, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[2616, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [45]:
reg_expiriment = setup(train_df, target = 'rating')
lr_model = create_model('lr', n_jobs = .01)
tuned_lr = tune_model(lr_model, optimize = 'RMSE')
test_predictions = predict_model(tuned_lr, data = test_df)

,Description,Value
0,Session id,8577
1,Target,rating
2,Target type,Regression
3,Original data shape,"(70002, 36)"
4,Transformed data shape,"(70002, 36)"
5,Transformed train set shape,"(49001, 36)"
6,Transformed test set shape,"(21001, 36)"
7,Numeric features,30
8,Categorical features,5
9,Rows with missing values,2.0%


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.1513,0.0370,0.1923,0.1889,0.1198,0.3228
1,0.1533,0.0388,0.1969,0.1634,0.1239,0.3557
2,0.1517,0.0378,0.1945,0.1685,0.1224,0.3466
3,0.1509,0.0370,0.1924,0.1746,0.1203,0.3361
4,0.1483,0.0362,0.1903,0.1680,0.1192,0.3276
5,0.1496,0.0363,0.1906,0.1823,0.1191,0.3237
6,0.1522,0.0374,0.1933,0.1782,0.1207,0.3366
7,0.1512,0.0371,0.1927,0.1770,0.1202,0.3301
8,0.1507,0.0369,0.1921,0.1790,0.1201,0.3318


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.1513,0.0370,0.1923,0.1890,0.1198,0.3228
1,0.1533,0.0388,0.1969,0.1634,0.1239,0.3557
2,0.1517,0.0378,0.1945,0.1685,0.1224,0.3465
3,0.1509,0.0370,0.1924,0.1746,0.1203,0.3361
4,0.1483,0.0362,0.1903,0.1680,0.1192,0.3276
5,0.1495,0.0363,0.1906,0.1824,0.1191,0.3237
6,0.1522,0.0374,0.1933,0.1782,0.1207,0.3366
7,0.1512,0.0371,0.1927,0.1770,0.1202,0.3301
8,0.1507,0.0369,0.1921,0.1790,0.1201,0.3318


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 2 candidates, totalling 20 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [46]:
test_df['rating'] = test_predictions['prediction_label']
test_df['movieId'] = test_df['movieId'].astype(int).astype(str)
test_df['userId'] = test_df['userId'].astype(int).astype(str)
test_df['userId_movieId'] = test_df['userId'] + '_' + test_df['movieId']
cols = ['userId_movieId', 'rating']
test_df = test_df[cols]
test_df.head()

,userId_movieId,rating
0,469_2124,0.667665
1,439_3753,0.683956
2,522_1682,0.794711
3,429_1217,0.850871
4,71_1210,0.813394


In [47]:
test_df.to_csv('submission.csv', index = False)